# Burseras and Friends
## How´s the environment ?
## Which are their friends ? 

In this example we will explore how to query and select a Node (Bursera, Node Type : Family) in the Tree Of Life.
We will extract the associated Cells and explore which kind of other taxa share that space. 
Let´s start!

![Burcera Environment](http://www.geographylists.com/mex_pv_forest.jpg "Bursera and its environment")


### Import modules libraries and load ploting style

In [33]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph

### Select the Node Bursera

Bursera is a Family of Plants. We want to select a Node in the Tree Of Life that is of the type Family and the name starts 
with "Burser". The expression "_.name=~STRING" means that we will look into the attribute "name" using a regular expression 
"=~" followed by any number of characters ".*"

In [34]:
bursera = list(Family.select(graph).where("_.name=~'Burser.*'")).pop()

The result is a list. Given that there is only one Node of the type Family that has the name "Burser" we can extract
the only element of the list using the standard method pop().

In [35]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

### Inspect taxonomy

In [36]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

In [37]:
bursera.getParent()

<TreeNode type: Order id = 933 name: Sapindales>

In [38]:
bursera.getParent().getParent().getParent().getParent()

<TreeNode type: Kingdom id = 6 name: Plantae>

The Tree Node is an object and has defined methods (refer to the documentation). One of this methods extract the cells in the spatial lattice that has associated Bursera occurrences. 

In [39]:
cells_with_burseras = bursera.cells

The result is a generator object. This is helpfull when the number of cells is big and surpasses the capacity of the machine
or the analyst.

In [40]:
cells_with_burseras

<generator object __iter__ at 0x7f139127f140>

In this example we want to get only the first 10 cells.

In [41]:
cells_with_burseras = list(cells_with_burseras)

In [74]:
len(cells_with_burseras)

811

## *Oke Oke*!, 
Now we want to get the subtrees of Life constrainted by the geographical attribute of this cells.
For doing this, simply get the occurrences within each cell with the method *occurrencesHere()* and use this output as the constructor argument.

In [42]:
c = cells_with_burseras[0]

In [43]:
c.occurrencesHere()

[<Occurrence pk=540443>,
 <Occurrence pk=536897>,
 <Occurrence pk=538067>,
 <Occurrence pk=538060>,
 <Occurrence pk=552009>,
 <Occurrence pk=1686161>,
 <Occurrence pk=2198672>,
 <Occurrence pk=1900876>,
 <Occurrence pk=2153230>,
 <Occurrence pk=2144783>,
 <Occurrence pk=556012>,
 <Occurrence pk=553369>,
 <Occurrence pk=536899>,
 <Occurrence pk=1798523>]

## Random subselection of cells
The number of cells is very big. An option for processing is to select a random uniform set of indexes. We can achieve this with the following strategy.

1. make a random sample selection


### Let's create a random subset of 100 cells.

In [44]:
import numpy.random as rnd

In [45]:
n = 100

In [46]:
indices = rnd.randint(0,len(cells_with_burseras),n)

In [47]:
selected_cells = [ cells_with_burseras[i] for i in indices ]

### Get the subtrees within the selected cells

Let's create first a Constructor function

In [48]:
ToTree = lambda cell : TreeNeo(cell.occurrencesHere())

and map it to the selected cells

In [49]:
%time trees = map(ToTree,selected_cells)

CPU times: user 3min 28s, sys: 5.05 s, total: 3min 33s
Wall time: 4min 36s


#### Uff! it took 3 minutes, but it's a lot of data

In [50]:
trees

[<LocalTree Of Life | Root: LUCA - n.count : 484- >,
 <LocalTree Of Life | Root: LUCA - n.count : 12- >,
 <LocalTree Of Life | Root: LUCA - n.count : 571- >,
 <LocalTree Of Life | Root: LUCA - n.count : 7- >,
 <LocalTree Of Life | Root: LUCA - n.count : 35- >,
 <LocalTree Of Life | Root: LUCA - n.count : 23- >,
 <LocalTree Of Life | Root: LUCA - n.count : 46- >,
 <LocalTree Of Life | Root: LUCA - n.count : 5- >,
 <LocalTree Of Life | Root: LUCA - n.count : 30- >,
 <LocalTree Of Life | Root: LUCA - n.count : 13- >,
 <LocalTree Of Life | Root: LUCA - n.count : 250- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTree Of Life | Root: LUCA - n.count : 31- >,
 <LocalTree Of Life | Root: LUCA - n.count : 133- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2632- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1716- >,
 <LocalTree Of Life | Root: LUCA - n.count : 15- >,
 <LocalTree Of Life | Root: LUCA - n.count : 27- >,
 <LocalT

#### Get the UNION of the these 100 trees~

In [51]:
big_tree = reduce(lambda a,b : a + b , trees)

INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging Trees
INFO Merging

In [52]:
big_tree.richness

25912

I told you, loads of data.
We can explore the content of this tree accessing to the attributes that start with the suffix ''to_''

In [53]:
mushrooms = big_tree.to_Fungi

Mushrooms is a TreeNode, i.e. not a complete TreeNeo object. To cast it (convert) to a TreeNode, and explore the families,classes, etc within it we must *plant-it* .

In [54]:
m = mushrooms.plantTreeNode()

In [55]:
type(mushrooms)

drivers.tree_builder.LocalTree

In [56]:
type(m)

drivers.tree_builder.TreeNeo

In [57]:
m.classes

[<TreeNode | Class: Eurotiomycetes - n.count : 5- >,
 <TreeNode | Class: Lichinomycetes - n.count : 2- >,
 <TreeNode | Class: Sordariomycetes - n.count : 1- >]

In [58]:
big_tree.to_Protozoa

<TreeNode | Kingdom: Protozoa - n.count : 4- >

In [59]:
big_tree.to_Plantae

<TreeNode | Kingdom: Plantae - n.count : 7701- >

In [60]:
big_tree.to_Animalia

<TreeNode | Kingdom: Animalia - n.count : 18197- >

### Cnidaria in the "terrestrial" environment ?  

In [67]:
cnidaria = big_tree.to_Animalia.to_Cnidaria

In [68]:
cnidaria

<TreeNode | Phylum: Cnidaria - n.count : 5- >

Where is this ?

In [69]:
cnidaria_cells = cnidaria.getExactCells()

In [70]:
cnidaria_cells

[<Cell id=223271>, <Cell id=199456>]

In [71]:
c1 , c2 = cnidaria_cells

In [72]:
c1.cell

u'MULTIPOLYGON (((-110.2690634159987 24.18688980100021, -110.2200634159987 24.18688980100021, -110.2200634159987 24.13788980100021, -110.2690634159987 24.13788980100021, -110.2690634159987 24.18688980100021)))'

![Cnidaria cell 1](cnidaria_cell1.png)

In [66]:
# c2.cell
## Dont do it, name to big.

![Cnidaria cell 2](cnidaria_cell2.png)

Close to the sea, but a little bit off the shore.

In [ ]:
## 

In [75]:
beatle = big_tree.to_Animalia.to_Arthropoda.to_Insecta.to_Coleoptera

## How many of these have beatles ?

In [76]:
with_beatles = filter(lambda tree : tree.hasNode(beatle),trees)

In [78]:
len(with_beatles)

12

# Environmental Exploration
Remember that the big tree is defined in the geographical space given by the selected cells, i.e. the random sample.
We can get the environmental summary statistics of these cells with the following command.

In [79]:
%time bursera_env = big_tree.associatedData.getEnvironmentalVariablesCells(with_std=True)

In [80]:
bursera_env

{'MaxTemperature_mean': 14.17116402116403,
 'MaxTemperature_std': 4.116918754484438,
 'MeanTemperature_mean': 21.776164021164046,
 'MeanTemperature_std': 3.459773822079493,
 'MinTemperature_mean': 29.385740740740758,
 'MinTemperature_std': 3.228895786792082,
 'Precipitation_mean': 55.899232804232774,
 'Precipitation_std': 38.096175261050234,
 'SolarRadiation_mean': 18550.605740740742,
 'SolarRadiation_std': 1027.6193796354646,
 'Vapor_mean': 1.6339417989417988,
 'Vapor_std': 0.5585689252847315,
 'WindSpeed_mean': 2.3305291005291005,
 'WindSpeed_std': 0.6210516806588766}

### The same for point data

In [81]:
only_bursera = big_tree.to_Plantae.to_Magnoliophyta.to_Magnoliopsida.to_Sapindales.to_Burseraceae

In [82]:
pnts_env_burs = only_bursera.associatedData.getEnvironmentalVariablesPoints()

In [83]:
pnts_env_burs

,MaxTemperature_mean,MaxTemperature_std,MeanTemperature_mean,MeanTemperature_std,MinTemperature_mean,MinTemperature_std,Precipitation_mean,Precipitation_std,Vapor_mean,Vapor_std,SolarRadiation_mean,SolarRadiation_std,WindSpeed_mean,WindSpeed_std
0,13.166667,4.079079,21.333333,3.659083,29.833333,3.435921,14.750000,17.440972,1.666667,0.623610,19022.750000,3321.711309,2.583333,0.493007
1,10.583333,5.040806,18.500000,4.406435,26.416667,4.030267,10.750000,8.001302,1.416667,0.493007,19243.416667,4598.550015,2.416667,0.493007
2,10.583333,5.040806,18.500000,4.406435,26.416667,4.030267,10.750000,8.001302,1.416667,0.493007,19243.416667,4598.550015,2.416667,0.493007
3,10.916667,2.871072,19.333333,2.527625,27.666667,2.748737,61.916667,70.374544,1.333333,0.471405,18422.750000,2304.748798,2.000000,0.000000
4,14.250000,4.832615,22.750000,4.380354,31.083333,3.839669,19.916667,25.381943,1.833333,0.687184,18657.333333,3183.815696,2.750000,0.433013
5,14.250000,5.246030,21.666667,5.071708,29.250000,5.068284,16.333333,17.182032,1.416667,0.493007,18787.583333,3722.641008,2.250000,0.433013
6,15.000000,5.016639,23.083333,4.974239,31.000000,4.618802,19.166667,24.309920,1.750000,0.721688,18930.166667,3571.761676,3.000000,0.000000
7,15.000000,5.016639,23.083333,4.974239,31.000000,4.618802,19.166667,24.309920,1.750000,0.721688,18930.166667,3571.761676,3.000000,0.000000
8,9.583333,2.722080,18.000000,2.645751,26.083333,2.752524,45.750000,42.575668,1.000000,0.000000,18881.833333,2361.063632,2.166667,0.372678
9,15.333333,5.405758,21.000000,5.446712,26.583333,5.663602,11.666667,11.374924,1.750000,0.829156,18753.500000,4365.401232,3.000000,0.000000


## General aproach

In [52]:
type(big_tree.levels)

list

In [45]:
birds = t.to_Animalia.to_Chordata.to_Aves

In [46]:
bursera

<TreeNode type: Family id = 343770 name: Burseraceae>

In [47]:
birds

<TreeNode | Class: Aves - n.count : 8- >

In [48]:
trees

[<LocalTree Of Life | Root: LUCA - n.count : 14- >,
 <LocalTree Of Life | Root: LUCA - n.count : 4- >,
 <LocalTree Of Life | Root: LUCA - n.count : 37- >,
 <LocalTree Of Life | Root: LUCA - n.count : 373- >,
 <LocalTree Of Life | Root: LUCA - n.count : 5- >,
 <LocalTree Of Life | Root: LUCA - n.count : 9- >,
 <LocalTree Of Life | Root: LUCA - n.count : 2- >,
 <LocalTree Of Life | Root: LUCA - n.count : 59- >,
 <LocalTree Of Life | Root: LUCA - n.count : 947- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1052- >]

In [49]:
filter(lambda tree : tree.hasNode(birds),trees)

[<LocalTree Of Life | Root: LUCA - n.count : 14- >,
 <LocalTree Of Life | Root: LUCA - n.count : 373- >,
 <LocalTree Of Life | Root: LUCA - n.count : 947- >,
 <LocalTree Of Life | Root: LUCA - n.count : 1052- >]

In [56]:
getNeighbourhood = lambda tree : tree.getNeighboringTrees()